## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [9]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [10]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예시(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 모으는 거죠.\n\n2. **모델 설계**: 그 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하려고 해요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 모르기 때문에, 입력(예: 사진)을 넣으면 잘못된 답을 내놓을 수 있어요.\n   - 이때, 정답(예: '이것은 고양이입니다')와 비교해서 얼마나 틀렸는지(오차)를 계산합니다.\n   - 그리고 이 오차를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정합니다. 이 과정을 반복하면서 모델은 점점 더 정확하게 사진이 고양이인지 강아지인지 구별하게 돼요.\n\n4. **반복 학습**: 이 과정을 여러 번 반복하면서 모델은 점점 더 좋아지고, 새로운 데이터에 대해서도 잘 맞출 수 있게 됩니다.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정입니다. 이렇게 학습을 통해서 우리가 원하는 일을 수행할 수 있게 되는 거예요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 342, 'prompt_tokens': 22, 'total_tokens': 364, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, '

아래는 스트리밍을 출력하는 예시 입니다.

In [11]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모읍니다. 예를 들어, 고양이 사진을 많이 보여주는 것처럼요.

2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있도록 수학적인 구조(모델)를 만듭니다. 이 구조는 사람의 뇌처럼 정보를 처리하는 방식과 비슷하게 만들어집니다.

3. **학습 과정**:
   - **예측하기**: 모델은 처음에는 무작위로 예측을 합니다. 예를 들어, 사진이 고양이인지 강아지인지 맞추려고 시도하는 거죠.
   - **오차 계산**: 예측이 틀리면, 얼마나 틀렸는지(오차)를 계산합니다.
   - **수정하기**: 이 오차 정보를 이용해서 모델의 내부 값(파라미터)을 조금씩 조정합니다. 이렇게 해서 다음번에는 더 정확하게 예측할 수 있도록 하는 거예요.
   
4. **반복 학습**: 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 데이터를 이해하고 예측할 수 있게 됩니다.

5. **완성**: 충분히 학습이 되면, 모델은 새로운 데이터(예를 들어, 새로운 고양이 사진)가 들어왔을 때도 잘 맞출 수 있게 됩니다.

요약하자면, 인공지능 모델은 많은 데이터를 보고, 예측을 하고, 틀리면 고치면서 점점 더 똑똑해지는 과정입니다. 이 과정을 통해 우리가 원하는 일을 잘 수행할 수 있게 되는 거죠!

### 출력파서(Output Parser)

#### 출력파서 StrOutputParser는 LLM이 생성한 출력 결과를 필요에 맞게 가공하고 구조화하는 단계입니다.


In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [13]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [14]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.\n\n2. **모델 설계**: 그 다음, 이 데이터를 이해하고 학습할 수 있도록 인공지능의 구조(모델)를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 여러 층으로 이루어진 '신경망'일 수 있어요.\n\n3. **학습 과정**:\n   - 인공지능은 처음에 무작위로 예측하거나 판단을 해요.\n   - 그리고 그 예측이 맞았는지 틀렸는지 평가하는 '손실 함수'라는 것을 계산해요.\n   - 틀렸다면, 그 틀림의 정도를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정해요.\n   - 이 과정을 수천, 수만 번 반복하면서 점점 더 정확하게 예측하게 되는 거죠.\n\n4. **최적화**: 이때 '경사 하강법' 같은 방법을 사용해서, 손실을 최소화하는 방향으로 가중치를 계속 조정해요.\n\n5. **완성**: 충분히 학습이 되면, 인공지능은 새로운 데이터(예를 들어, 새로운 고양이 사진)가 들어와도 잘 구별하거나 판단할 수 있게 돼요.\n\n요약하자면, 인공지능은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정이라고 할 수 있어요!"

In [15]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.

2. **모델 설계**: 그 다음, 이 데이터를 이해하고 학습할 수 있도록 인공지능의 구조(모델)를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 생각할 수 있어요.

3. **학습 과정**:
   - 인공지능은 처음에는 무작위로 예측하거나 판단을 해요.
   - 그리고 실제 정답(예를 들어, 사진이 고양이인지 아닌지)을 알려줍니다.
   - 인공지능은 자신의 예측이 틀렸다는 것을 알고, 틀린 정도(오차)를 계산해요.
   - 이 오차를 줄이기 위해 내부의 '가중치'라는 값을 조정합니다.
   
4. **반복 학습**:
   - 이 과정을 여러 번 반복하면서, 인공지능은 점점 더 정확하게 예측할 수 있게 돼요.
   - 마치 연습을 많이 해서 실력이 늘어나는 것과 같아요.

5. **완성**:
   - 충분히 학습하면, 인공지능은 새로운 데이터(처음 본 적 없는 사진 등)도 잘 판단할 수 있게 됩니다.

요약하자면, 인공지능은 많은 데이터를 보고, 틀린 점을 고치면서 점점 더 똑똑해지는 과정이라고 할 수 있어요!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [16]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [17]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [18]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
Customer: Hello! Just one person. I'd like to order, please.
Waiter: Absolutely. Would you like to see the menu?
Customer: Yes, please. Thank you.
Waiter: Here you go. Take your time. Would you like a drink to start?
Customer: Yes, I’ll have a glass of water and the chicken pasta, please.
Waiter: Great choice. I'll bring those out for you shortly.  
Customer: Thank you very much.

- 한글 해석:
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?
손님: 안녕하세요! 한 명이에요. 주문하고 싶어요.
웨이터: 알겠습니다. 메뉴를 보여드릴까요?
손님: 네, 부탁해요. 감사합니다.
웨이터: 여기 있습니다. 천천히 보세요. 음료부터 드시겠어요?
손님: 네, 물 한 잔과 치킨 파스타로 주세요.
웨이터: 좋은 선택이세요. 바로 가져다드리겠습니다.
손님: 매우 감사합니다.


In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello, I'd like to order, please. Could I see the menu?"  
- 한글 해석:  
"안녕하세요, 주문하고 싶어요. 메뉴판을 볼 수 있을까요?"

In [20]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
**Customer:** Hi, I’d like to order a large pepperoni pizza, please.  
**Staff:** Sure! Would you like any drinks or sides with that?  
**Customer:** Yes, just a soda. Can I also get some extra cheese on the pizza?  
**Staff:** Absolutely. Your total will be $20. Would you like it for delivery or pickup?  
**Customer:** For delivery, please. How long will it take?  
**Staff:** About 30 minutes. Can I have your delivery address?  
**Customer:** Yes, it’s 123 Main Street. Thank you!  
**Staff:** Thank you! Your order is on its way.  

- 한글 해석:  
**손님:** 안녕하세요, 대형 페퍼로니 피자 하나 주문하고 싶어요.  
**직원:** 네, 알겠습니다! 음료수나 사이드 메뉴도 같이 드릴까요?  
**손님:** 네, 탄산음료 하나 주세요. 그리고 피자에 치즈를 좀 더 넣어도 될까요?  
**직원:** 물론이죠. 총 금액은 20달러입니다. 배달받으시겠어요 아니면 포장하실 건가요?  
**손님:** 배달로 주세요. 얼마나 걸리나요?  
**직원:** 약 30분 정도 걸릴 거예요. 배달 주소를 알려주시겠어요?  
**손님:** 네, 주소는 123 메인 스트리트입니다. 감사합니다!  
**직원:** 감사합니다! 주문이 준비되어 배달될 거예요.